PyTorch提供了两个主要的特性：
1. 类似于ndarray的Tensor，但它可以在GPU上进行计算
2. 自动微分帮助我们训练NN

下面的例子用一个包含四个参数的多项式来拟合y=sin(x)。损失函数使用欧式距离。

# Tensor

## Warm-up: numpy

numpy只提供基础的矩阵运算，不涉及深度学习，梯度下降。下面我们用numpy手动通过前向和反向传播实现上面的拟合任务。

In [1]:
import numpy as np
import math

In [2]:
# 生成一些输入输出数据
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

In [3]:
# 随机初始化参数
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

In [4]:
learning_rate = 1e-6
for t in range(20000):
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    loss = np.square(y_pred - y).sum()
    if t % 3000 == 2999:
        print(t, loss)
    
    # 用梯度下降法计算loss对参数的导数
    tmp = 2.0 * (y_pred - y)
    grad_a = tmp.sum()
    grad_b = (tmp * x).sum()
    grad_c = (tmp * x**2).sum()
    grad_d = (tmp * x**3).sum()
    
    # 更新参数
    a -= grad_a * learning_rate
    b -= grad_b * learning_rate
    c -= grad_c * learning_rate
    d -= grad_d * learning_rate

print("y = {a} + {b}x + {c}x^2 + {d}x^3".format(a=a, b=b, c=c, d=d))

2999 8.82190653581357
5999 8.817165451301104
8999 8.81716541000788
11999 8.817165410007025
14999 8.817165410007025
17999 8.817165410007027
y = -1.7185327486745663e-16 + 0.8567408430737578x + 2.4860231713332116e-17x^2 + -0.09333038904059505x^3


## PyTorch: Tensor

为了让Tensor能够在GPU上计算，需要为其指定正确的device。下面用Tensor来实现上面的功能。

In [5]:
import torch

dtype = torch.float
device = torch.device("cpu")  # 在cpu上运行
device = torch.device("cuda:0")  # 在gpu上运行

x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(20000):
    # forward pass
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    # loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 3000 == 2999:
        print(t, loss)
    # backprop
    tmp = 2.0 * (y_pred - y)

    grad_a = tmp.sum()
    grad_b = (tmp * x).sum()
    grad_c = (tmp * x**2).sum()
    grad_d = (tmp * x**3).sum()
    
    # 更新参数
    a -= grad_a * learning_rate
    b -= grad_b * learning_rate
    c -= grad_c * learning_rate
    d -= grad_d * learning_rate
    
print("y = {a} + {b}x + {c}x^2 + {d}x^3".format(a=a, b=b, c=c, d=d))        

2999 8.839166641235352
5999 8.817167282104492
8999 8.817166328430176
11999 8.817166328430176
14999 8.817166328430176
17999 8.817166328430176
y = -2.0428718716658523e-09 + 0.8567265868186951x + -1.1004080313625764e-08x^2 + -0.09332836419343948x^3


# Autograd

## PyTorch: Tensor and autograd

如果网络非常复杂，自己去计算loss对参数的导数就会变得很麻烦。PyTorch中的autograd包可以帮助我们在反向传播的过程中实现自动求导。使用autograd时，正向传播过程定义了一个计算图，图中 的节点都是Tensor，边则是表示输入与输出之间关系的函数。反向传播时可以轻松计算出梯度信息。对于一个Tensor a，如果将他的requires_grad指定为True，则x.grad是另一个Tensor，存储了a的梯度。下面我们就用自动求导功能实现上述代码，无须再手动执行反向传播的过程了。

In [6]:
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")

# 默认情况下requires_grad是False，表示在反向传播时我们不需要求loss对它的导数
x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype, device=device)
y = torch.sin(x)

a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(20000):
    
    # forward pass
    y_pred = a + b * x + c * x ** 2 + d * x ** 3
    
    # loss
    loss = (y_pred - y).pow(2).sum()
    
    if t % 3000 == 2999:
        print(t, loss)
    
    # 反向传播时通过调用loss.backward()，自动计算loss对所有requires_grad=True的张量的梯度
    # 即这里的 a，b，c，d。它们的梯度值保存在 .grad属性中。
    loss.backward()
    
    # 手动更新参数
    # 更新参数的操作不需要追踪梯度，因此把它写在torch.no_grad()中，一定不要忘记
    with torch.no_grad():
        a -= learning_rate * a.grad  # 这里不能写成 a = a- ...，a需要执行in-place运算
        b -= learning_rate * b.grad 
        c -= learning_rate * c.grad 
        d -= learning_rate * d.grad 

        # 手动清空梯度缓存，准备下次迭代
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None
    
print("y = {a} + {b}x + {c}x^2 + {d}x^3".format(a=a, b=b, c=c, d=d))        

2999 tensor(8.8570, grad_fn=<SumBackward0>)
5999 tensor(8.8172, grad_fn=<SumBackward0>)
8999 tensor(8.8172, grad_fn=<SumBackward0>)
11999 tensor(8.8172, grad_fn=<SumBackward0>)
14999 tensor(8.8172, grad_fn=<SumBackward0>)
17999 tensor(8.8172, grad_fn=<SumBackward0>)
y = 3.940505699517871e-09 + 0.8567265868186951x + -1.1153288959064867e-08x^2 + -0.09332836419343948x^3


## PyTorch: Defining new autograd functions

Torch中所定义的操作f都带有autograd的功能。即只要Tensor的requires_grad设为True，所有涉及该Tensor的操作都能在loss.backward()之后将经过当前操作的梯度向前传递过去。即 grad_in = grad_f(grad_out)，in和out分别是操作f的输入和输出，grad_in是loss对in的梯度，grad_out是loss对out的梯度，grad_f是f的out对in的导数，即链式法则，这是很好理解的。

PyTorch中不可能定义了我们所需要的所有操作，但是我们可以很容易地在PyTorch中自定义带有autograd功能的操作。方法是继承torch.autograd.Function并且实现forward和backward方法。之后在定义计算图时，我们就可以通过实例化并像函数调用一样使用这个自定义的操作了，将Tensor作为输入传入。下面我们自定义个带有autograd功能的三阶勒让德多项式操作：P3(x) = 0.5 * (5x^2 - 3x)。计算图为 y = b * P3(c+dx)。 

In [7]:
class LegendrePolynomial3(torch.autograd.Function):
    """
    我们可以通过继承torch.autograd.Function自己实现一个autograd函数
    """
    @staticmethod
    def forward(ctx, input):
        """
        前向传播时接受一个输入Tensor，返回一个输出Tensor。
        ctx是一个与上下文相关的对象，存储用于反向传播的各种信息。
        使用ctx.save_for_backward可以存储任意用于反向传播的对象。
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * x ** 3 - 3 * x)  # 返回三阶勒让德多项式的结果 
    
    @staticmethod
    def backward(ctx, grad_output):
        """
        反向传播接受一个Tensor，保存了loss函数对该勒让德多项式输出地方的导数grad_output。
        现在我们需要计算loss对该勒让德多项式输入处的导数，让后继续回传给下一个运算环节。
        """
        input, _ = cts.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)

In [8]:
# dtype = torch.float
# device = torch.device("cpu")

# # x，y默认的requires_grad为False
# x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
# y = torch.sin(x)

# a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)  # 返回指定大小，值为fill_value的矩阵 
# b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)  # 返回指定大小，值为fill_value的矩阵 
# c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)  # 返回指定大小，值为fill_value的矩阵 
# d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)  # 返回指定大小，值为fill_value的矩阵 

# learning_rate = 5e-6
# for t in range(2000):
#     # 假设勒让德多项式操作类中包含一个apply方法，返回一个callable的对象(相当于函数)，取别名为P3
#     # 此时P3会使用我们自定义的autograd操作
#     P3 = LegendrePolynomial3.apply  
#     y_pred = a + b + P3(c + d * x)  # c+dx为传入的参数
    
#     loss = (y_pred - y).pow(2).sum()
#     if t % 100 == 99:
#         print(t, loss.item())
    
#     loss.backward()
    
#     with torch.no_grad():
#         a -= learning_rate * a.grad  # 这里不能写成 a = a- ...，a需要执行in-place运算
#         b -= learning_rate * b.grad 
#         c -= learning_rate * c.grad 
#         d -= learning_rate * d.grad 

#         # 手动清空梯度缓存，准备下次迭代
#         a.grad = None
#         b.grad = None
#         c.grad = None
#         d.grad = None

# nn module

## PyTorch: nn

有了autograd，我们可以通过连接各种基础操作定义一些计算图，并实现自动求导。但是对于大规模神经网络，这太底层了。PyTorch中的nn模块实现了对诸多模块更高层的封装，并保存中间所有可训练参数的信息。nn模块也定义了很多使用的loss函数。下面我们就用nn模块实现一个对多项式参数的求取，该多项式为 y = a*x + b*x^2 + c*x^3。

In [9]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

p = torch.tensor([1, 2, 3])  # (3,)
xx = x.unsqueeze(-1).pow(p)  # (2000, 3) (x, x^2, x^3) 不同笔的数据永远在第一维度

In [10]:
# nn.Sequential包含一系列的Modules，它接受输入的张量，经过所有包含的Modules后输出新的张量
# 因为类中重写了__call__方法，因此返回的实例可以像函数一样被调用
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),  # (2000, 3) x (3, 1) = (2000, 1) 第i行表示 a*xi + b*xi^2 + c*xi^3
    torch.nn.Flatten(0, 1)  # 将前两个维度(第0维和第1维)打开
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    # backward之前一定不要忘记清空梯度缓存
    model.zero_grad()
    
    # 计算loss对model中所有可训练参数的梯度
    loss.backward()
    
    # 更新参数
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# 训练完成，可以像查看list中的元素一样查看第某层的信息
linear_layer = model[0]  # 第一层的信息

# 查看weight和bias
print(linear_layer.bias)  # 只取值的话还需要加上item()
print(linear_layer.weight)

99 164.63064575195312
199 111.96536254882812
299 77.10670471191406
399 54.03215026855469
499 38.75706481933594
599 28.644275665283203
699 21.948598861694336
799 17.51502799987793
899 14.57902717590332
999 12.634567260742188
1099 11.346598625183105
1199 10.493450164794922
1299 9.928211212158203
1399 9.553703308105469
1499 9.305509567260742
1599 9.14100456237793
1699 9.031965255737305
1799 8.959668159484863
1899 8.911718368530273
1999 8.879920959472656
Parameter containing:
tensor([-0.0020], requires_grad=True)
Parameter containing:
tensor([[ 8.4926e-01,  3.4305e-04, -9.2266e-02]], requires_grad=True)


## PyTorch: optim

目前为止我们都是在torch.no_grad中手动更新的参数。因为上面的示例用的是最基本的SGD优化器。对于更复杂优化器，比如Adam，RMSProp等，手动更新参数的过程就太复杂了。torch.optim模块中包含了所有常用的优化器，封装了它们的逻辑。

In [11]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

p = torch.tensor([1, 2, 3])  # (3,)
xx = x.unsqueeze(-1).pow(p)  # (2000, 3) (x, x^2, x^3)

model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),  # (2000, 3) x (3, 1) = (2000, 1) 第i行表示 a*xi + b*xi^2 + c*xi^3
    torch.nn.Flatten(0, 1)  # 将前两个维度(第0维和第1维)打开
)

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6

# 定义优化器，传入第一个参数是该网络所有需要优化的参数
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for t in range(2000):
    y_pred = model(xx)
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    # backward之前一定不要忘记清空梯度缓存
    optimizer.zero_grad()
    
    # 计算loss对model中所有可训练参数的梯度
    loss.backward()
    
    # 更新参数
    optimizer.step()

# 训练完成，可以像查看list中的元素一样查看第某层的信息
linear_layer = model[0]  # 第一层的信息

# 查看weight和bias
print(linear_layer.bias)  # 只取值的话还需要加上item()
print(linear_layer.weight)

99 9156.8232421875
199 9149.6064453125
299 9143.017578125
399 9136.638671875
499 9130.294921875
599 9123.974609375
699 9117.6630859375
799 9111.359375
899 9105.064453125
999 9098.775390625
1099 9092.498046875
1199 9086.228515625
1299 9079.9658203125
1399 9073.7119140625
1499 9067.466796875
1599 9061.228515625
1699 9055.0
1799 9048.779296875
1899 9042.56640625
1999 9036.36328125
Parameter containing:
tensor([0.1389], requires_grad=True)
Parameter containing:
tensor([[ 0.0384, -0.4749,  0.0582]], requires_grad=True)


## PyTorch: Custom nn Modules

上面通过torch.nn.Sequential的方式定义只能定义顺序序列。对于更为复杂的模型，我们需要自定义网络类并继承nn.Module。同时还要定义forward描绘输入与输出之间的逻辑。

In [12]:
class Polynomial3(torch.nn.Module):
    def __init__(self):
        super().__init__()
        """
        在构造函数中定义所有需要优化的参数。
        Parameters是Tensor的子类，当用于Module中时有一些特殊的特性：
        如果它被指定为Module的成员属性后会被自动添加到model的参数列表中：parameters()，
        指定普通的Tensor则没有这种效果，不会缓存它的状态。
        If there was no such class as Parameter, these temporaries would get registered too.
        """
        self.a = torch.nn.Parameter(torch.randn(()))  # Parameter的两个参数分别时data，requires_grad=True
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
    
    def forward(self, x):
        """
        描绘网络输入与输出之间的关系。可以使用构造函数中定义的模块。
        """
        return self.a + self.b * x + self.c * x**2 + self.d * x**3

In [13]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

model = Polynomial3()
criterion = torch.nn.MSELoss(reduction='sum')
learning_rate = 1e-6
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for t in range(2000):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 22678.06640625
199 22658.73046875
299 22641.072265625
399 22623.923828125
499 22606.93359375
599 22590.013671875
699 22573.107421875
799 22556.205078125
899 22539.31640625
999 22522.43359375
1099 22505.55859375
1199 22488.693359375
1299 22471.833984375
1399 22454.98828125
1499 22438.146484375
1599 22421.3125
1699 22404.48828125
1799 22387.673828125
1899 22370.865234375
1999 22354.06640625


In [14]:
print(model.a.item())
list(model.parameters())

-0.8115254044532776


[Parameter containing:
 tensor(-0.8115, requires_grad=True),
 Parameter containing:
 tensor(-0.0622, requires_grad=True),
 Parameter containing:
 tensor(0.6445, requires_grad=True),
 Parameter containing:
 tensor(0.2350, requires_grad=True)]

## PyTorch: Control Flow + Weight Sharing

为了说明，下面我们实施一个比较奇怪的模型：一个3-5阶的多项式，y = a + b * x + c * x^2 + d * x^3 + (e * x^exp + ...)，其中exp是一个3-5之间的随机数，并且括号里执行exp次，且每一次都是复用同一个参数e。在定义这个计算图的时候，我们可以直接使用Python中的控制流，比如循环或条件语句。也就是说，如果我们想要在网络中复用某一个部分，完全可以使用Python的语言逻辑来安全、简单地实现。

In [15]:
import random

class DynamicNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))
    
    def forward(self, x):
        y = self.a + self.b * x + self.c * x**2 + self.d * x**3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

In [16]:
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

model = DynamicNet()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
criterion = torch.nn.MSELoss(reduction='sum')

for t in range(30000):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

1999 1000.8871459960938
3999 448.5474548339844
5999 196.77584838867188
7999 99.22183990478516
9999 49.192604064941406
11999 26.75980567932129
13999 16.95676040649414
15999 12.462722778320312
17999 10.449708938598633
19999 9.54977035522461
21999 9.153962135314941
23999 8.825005531311035
25999 8.905901908874512
27999 8.675395965576172
29999 8.604968070983887


In [17]:
print(model.a.item())
print(model.b.item())
print(model.c.item())
print(model.d.item())
print(model.e.item())

-0.0019167063292115927
0.8544856309890747
-0.00019232860358897597
-0.09328288584947586
0.00010245972953271121
